### Tutorial: Merge LoRA Adapters

HME uses a multi-stage training strategy, which often requires merging previous stage adapters into the base model to serve as the starting point for the next stage or for efficient inference.

**Why Merge?**
1. **Inference Efficiency:** Merged models have lower latency than loading base + adapter separately.
2. **Dependency Chain:** Some downstream tasks (like QA) are fine-tuned on top of the comprehension-pretrained model. You must create the `_merged` version of the parent model first.

**Workflow:**
This notebook allows you to define the Base Model and Adapter, then performs the merge operation.

In [5]:
import sys
import os
from pathlib import Path
from dataclasses import dataclass

# Add project root to path
sys.path.append('..')

# Import core merging logic
from hme.merge import merge_lora_model

# Directories
CHECKPOINTS_DIR = Path('../checkpoints')
BASE_MODEL_NAME = "Meta-Llama-3-8B-Instruct"  # Change this if your base model folder differs
BASE_MODEL_PATH = CHECKPOINTS_DIR / BASE_MODEL_NAME

if not BASE_MODEL_PATH.exists():
    print(f"Warning: Base model not found at {BASE_MODEL_PATH}. Please run 'download_checkpoints.ipynb' first.")

In [2]:
# Import the argument classes and the merge function
from hme.merge import merge_lora_model, ModelArguments, DataArguments

def run_merge(base_path, adapter_name, task_type=None):
    """
    Helper to prepare arguments and run the merge.
    """
    adapter_path = CHECKPOINTS_DIR / adapter_name
    # Auto-generate output path
    output_path = CHECKPOINTS_DIR / f"{adapter_name}_merged"
    
    # Check existence
    if not adapter_path.exists():
        print(f"Skipping: Adapter {adapter_name} not found.")
        return None
        
    print(f"Merging: {base_path.name} + {adapter_name} -> {output_path.name}")
    
    # Instantiate arguments directly
    model_args = ModelArguments(
        model_name_or_path=str(base_path),
        peft_model_path=str(adapter_path),
        merged_model_path=str(output_path)
    )
    data_args = DataArguments(task_type=task_type)
    
    # Execute
    merge_lora_model(model_args, data_args)
    
    return output_path

### 1. Execute Merge Chain (Demo: General QA)

We will demonstrate the merge process using the **General QA** task. This is a multi-stage merge:
1. **Stage 1:** Merge `Comprehension Pretrain` adapter into the Base Llama-3 model.
2. **Stage 2:** Merge `General QA` adapter into the result of Stage 1.

> **Note:** For other tasks (e.g., Pocket-based Generation, Property QA), please refer to `scripts/merge_models.sh` for the complete dependency chains.

In [6]:
# --- Stage 1: Base -> Pretrain Merged ---
print("--- Stage 1: Merging Pretrain Adapter ---")
path_stage1 = run_merge(BASE_MODEL_PATH, "HME_comprehension-pretrain")

if path_stage1:
    # --- Stage 2: Pretrain Merged -> General QA Model ---
    print("\n--- Stage 2: Merging General QA Adapter ---")
    run_merge(path_stage1, "HME_general-qa")
else:
    print("\n Stage 1 failed or skipped. Cannot proceed to Stage 2.")

Using vocab_file: vocab_800_other_tasks.txt to load fragment list.


--- Stage 1: Merging Pretrain Adapter ---
Merging: Meta-Llama-3-8B-Instruct + HME_comprehension-pretrain -> HME_comprehension-pretrain_merged
--- Loading Base Model and Tokenizer ---


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

--- Loading PEFT Adapter from ../checkpoints/HME_comprehension-pretrain ---
--- Merging Adapter into Base Model ---
--- Saving Merged Model to ../checkpoints/HME_comprehension-pretrain_merged ---
[2025-12-08 07:20:51,027] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/lvliuzhenghao/miniconda3/envs/mollama/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/lvliuzhenghao/miniconda3/envs/mollama/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


No task-specific head to save (task is likely conditional generation).
--- Model merged and saved successfully to ../checkpoints/HME_comprehension-pretrain_merged ---

--- Stage 2: Merging General QA Adapter ---
Merging: HME_comprehension-pretrain_merged + HME_general-qa -> HME_general-qa_merged
--- Loading Base Model and Tokenizer ---


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/lvliuzhenghao/llzh/NC_minor/HME/src/hme/util.py:296: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint_path, "feature_fuser.pth"))


--- Loading PEFT Adapter from ../checkpoints/HME_general-qa ---
--- Merging Adapter into Base Model ---
--- Saving Merged Model to ../checkpoints/HME_general-qa_merged ---
No task-specific head to save (task is likely conditional generation).
--- Model merged and saved successfully to ../checkpoints/HME_general-qa_merged ---


### 2. Verify Output
Let's check the generated merged models.

In [ ]:
print("Merged models in checkpoints directory:")
for item in CHECKPOINTS_DIR.iterdir():
    if item.is_dir() and item.name.endswith("_merged"):
        print(f"{item.name}")

Merged models in checkpoints directory:
HME_comprehension-pretrain_merged
HME_general-qa_merged


: 